In [1]:
import pandas as pd
import dalmatian as dm

In [ ]:
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq")
terra_rnaseq_df = wm.get_samples()

In [ ]:
def load_rnaseqc(terra_path):
    rnaseqc_count_df = pd.read_csv(terra_path, sep='\t', skiprows=2)
    rnaseqc_count_df = rnaseqc_count_df.set_index(rnaseqc_count_df.apply(lambda x: f"{x[1]} ({x[0].split('.')[0]})", axis=1))
    rnaseqc_count_df = rnaseqc_count_df.drop(["Name", "Description"], axis=1)
    return rnaseqc_count_df

In [ ]:
from multiprocessing import Pool
pool = Pool(12)
rnaseqc_count_dfs = pool.map(load_rnaseqc, terra_rnaseq_df.rnaseqc2_gene_counts)

In [ ]:
rnaseqc_count_mat = pd.concat(rnaseqc_count_dfs, axis=1)
rnaseqc_count_mat = rnaseqc_count_mat.T
##all_samples_qc = tc.get(name='expression-869e', version=6, file='all_samples_qc')
all_samples_qc = tc.get(name='expression-d035', version=44, file='all_samples_qc')

In [ ]:
pool.close()

In [ ]:
rnaseqc_count_mat.to_csv("rnaseqc_count_mat.csv")

In [ ]:
update = True
if update:
   new_dataset_id = tc.update_dataset(
        dataset_id="rnqseqc2-gene-count-profile-5577",
        changes_description="rnqseqc2 count cds ids",
        upload_files=[
            {
                "path": "rnaseqc_count_mat.csv",
                "name": "rnqseqc2_gene_count_profile", # optional, will use file name if not provided
                "format": "NumericMatrixCSV", # or "NumericMatrixCSV" or "TableCSV"
                "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
            }
        ],
        upload_async=False,
        dataset_description="rnqseqc2 count profile",
   )
else:
   new_dataset_id = tc.create_dataset(
       "rnqseqc2_gene_count_profile",
       dataset_description="rnqseqc2 count profile",
       upload_files=[
           {
               "path": "rnaseqc_count_mat.csv",
               "name": "rnqseqc2_gene_count_profile", # optional, will use file name if not provided
               "format": "NumericMatrixCSV", # or "NumericMatrixCSV" or "TableCSV"
               "encoding": "utf-8" # optional (but recommended), will use iso-8859-1 if not provided
           }
       ],
       folder_id="a9eedc220a6a4e70b8f1e64d2e57ed87", # optional, will default to your home folder if not provided
   )

new_dataset_id